<a href="https://colab.research.google.com/github/NatyDV/Business-Analytics/blob/main/Reto2_%C3%81rbolesdeDecisi%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Reto 2- Árboles de Decisón**

0. Cargar las librerias de trabajo

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Se trae el modelo
from sklearn.tree import DecisionTreeClassifier #Prestamos Interbibliotecario

#Para llamar los archivos de Drivve- Sin tener que subirlos
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. Se acargan los datos desde el Drive

In [2]:
nxl='/content/drive/MyDrive/Colab Notebooks/2. BD2_In_App_Pur_Int.xlsx'
XDB=pd.read_excel(nxl,sheet_name=0)
XDB=XDB.dropna() #Evita celdas vacías
XDB.head()

#Seleccionar las variables de interés
XD=XDB[['Age','Income','Years','Hours']]
XD.head()
XD=np.array(XD)   #Convierto las variables en númeroa- Variables de Entrada

yd=XDB[['Buy']]; yd=np.array(yd)  #Varaibles de Salida- Convierto a la tabla

2. Se procede con la construcción del modelo de Árbol

In [3]:
from re import S
mar=DecisionTreeClassifier(criterion='gini',max_depth=4)
mar.fit(XD,yd)
ydp=mar.predict(XD)   #Estos valores de pronóstico sirven para evaluar modelo

#Evaluamos el modelo
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(yd,ydp)
VN=cm[0,0];FP=cm[0,1];FN=cm[1,0];VP=cm[1,1]

Ex=(VP+VN)/(VP+VN+FP+FN)
Se=VP/(VP+FN); Es=VN/(VN+FP)
print('Matriz de Confusión')
print(cm)

print('Exactitud',Ex)
print('Sensibilidad',Se)
print('Especificidad', Es)

Matriz de Confusión
[[139  50]
 [ 11 200]]
Exactitud 0.8475
Sensibilidad 0.9478672985781991
Especificidad 0.7354497354497355


3. Se procede con la visualización del arbol

In [4]:
from sklearn.tree import export_graphviz   #Estructura del Árbol
from pydotplus import graph_from_dot_data  #Que datos le voy a colocar al árbol
from IPython.display import Image
from google.colab import files

vs=['Age','Income','Years','Hours']      #Estos nombres aparaecen en el árbol

dot_data=export_graphviz(mar,feature_names=vs)  #Asegurar nombres del gráfico
graph=graph_from_dot_data(dot_data)             #Se hace el gráfico del árbol
graph.write_png('arbol.png')                    #Se crea la imagen
Image('arbol.png')
files.download('arbol.png')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

4. Le calculamos los valores de compra a los solicitantes nuevos

In [5]:
XDB2=pd.read_excel(nxl,sheet_name=1)
XDB2=XDB2 [['Age','Income','Years','Hours']]
XDB2=XDB2.dropna()

XD2=np.array(XDB2)
ydp2=mar.predict(XD2)
XDB2['Buy']=ydp2  #Le pego la columna a la base de datos
XDB2.head()

X=XDB2.to_excel('NuevosSolicitantes.xlsx')


len(ydp2)
np.sum(ydp2)  # Compradores
len(ydp2)-np.sum(ydp2)    #No compradores



1

**Análisis de resultados**
En la base de datos se puede observar un total de 400 personas divididos en 139 que no compraron en la aplicacion (47%) y un total de 211 si lo hicieron (53%). De acuerdo con lo anterior, el modelo arrojo un indice de exactitud cercano al 84.75%, lo que indica que los datos son muy precisos. Además la sensibilidad dió 94.7% indicando que el modelo es muy util para indicar cuantas personas compraron en la app. En cuanto a la especificidad el modelo arrojo un 73.5%, representando que es buena para analizar las personas no compradoras en la aplicación.         

En el primero nodo puro es de un ingreso menor o igual a a 63.5 (Icome<= 63.5), lo cual representa una probabilidad del 100% de que los jugadores tengan un igresos menor a este no realicen compras dentro de la aplicación. Por otro lado el segundo nodo puro tiene la regla de que la edad se mayor a 0.5 años (Icome>63.5 AND Years <=1.5 AND Years >0.5) y finalmente el tercer nodo puro tiene la regla de que la edad se mayor a 23 años (Icome>63.5 AND Years <= 1.5 AND Years <=0.5 AND Age >23). Para los nuevos casos las reglas son las siguientes:
1. Income>63.5 AND Years >1.5 AND Years > 5.5 AND Income >76.5
2. Income >63.5 AND Years <=1.5 AND Years >0.5 (28,0)

Segun esto Para el jugador 1 hay un porcentaje de (33%) de que no haga compras en la app y uno de (66%) de que lo haga. Asi mismo para el caso 2 hay un porcentaje de (100%) de que no haga compras en la app y un (0%) que si haga.